In [ ]:
cd /home/dcor/niskhizov/AdversarialRendering/mitsuba/gsoup/src

In [ ]:
import glob
import mitsuba as mi
import drjit as dr
import numpy as np
import cv2
from gsoup.image import (
    change_brightness,
    add_alpha,
    resize,
    tonemap_reinhard,
    linear_to_srgb,
)
import matplotlib.pyplot as plt

mi.set_variant(
    "cuda_ad_rgb"
    # "scalar_rgb"
)  # "llvm_ad_rgb", "scalar_rgb" # must set before defining the emitter


patchs = glob.glob('/home/dcor/niskhizov/AdversarialRendering/botorch_snapshots/*patch*')

cam_wh=(256, 256)
cam_fov=45
proj_wh=(256, 256)
proj_fov=45
ambient_color=list(np.array([0.1, 0.1, 0.1] )*10)
proj_brightness = 30

spp=256
proj_response_mode = "srgb"

import gsoup
from gsoup.projector_plugin_mitsuba import ProjectorPy
import mitsuba as mi
mi.register_emitter("projector_py", lambda props: ProjectorPy(props))


In [ ]:
# roughness = list(range(0.1,0.8, 0.1))
# metallics  = list(range(0, 1.0, 0.1))

roughness = list(np.arange(0.1, 0.8, 0.1))

metallics  = list(np.arange(0, 1.0, 0.1))


def generate_random_scene():
    """
    Generate a random scene dictionary for Mitsuba.
    """            
    random_patch = np.random.choice(patchs)

    roughnes =  np.random.choice(roughness)
    metallic = np.random.choice(metallics)

    proj_brightness = 5.0#np.random.uniform(1.0, 10.0)

    ambient_color_scale = np.random.uniform(0.1, 1.0)

    ambient_color=list(np.array([1.0,1.0,1.0] )*ambient_color_scale)

    focus_distance = np.random.uniform(1.0, 20.0)

    scene_dict = {

                    "type": "scene",

                    "integrator": {
                        "type": "path",
                        "hide_emitters": True,
                        "max_depth": 6,
                    },
                    "camera": {
                        "type": "thinlens",
                        'aperture_radius': 0.02,  # control DoF (0.0 = pinhole)
                        'focus_distance': focus_distance,  # meters to subject
                        "fov": cam_fov,
                        
                        "to_world": mi.ScalarTransform4f().look_at(
                            origin=[0.1, 0, 0.0],  # along +X axis
                            target=[0, 0, 0],
                            up=[0, 0, 1],  # Z-up
                        ),
                        "film": {
                            "type": "hdrfilm",
                            "width": cam_wh[0],
                            "height": cam_wh[1],
                            "pixel_format": "rgba",
                            "rfilter": {"type": "box"},
                        },
                        "sampler": {
                            "type": "independent",
                            "sample_count": spp,  # number of samples per pixel
                        },
                    },

                    "wall3": {
                        "type": "rectangle",
                        "to_world": mi.ScalarTransform4f()
                        .translate([-2.0, 0.0, 0.0])
                        .rotate([0, 1, 0], 90),
                        "bsdf": 
                        {

                        'type': 'principled',
                        'base_color': {
                            'type': 'bitmap',
                            'filename': f'{random_patch}',

                        },
                        'roughness' : roughnes,
                        'metallic': metallic,
                        },
                    },

                    "projector": {
                        "type": "projector_py",
                        "irradiance": {
                            # "type": "ref",
                            # "id": "proj_texture",
                            "type": "bitmap",
                            'filename': '/home/dcor/niskhizov/AdversarialRendering/mitsuba/mat_data/WoodFloor064_1K/WoodFloor064_1K-JPG_Color.jpg',
                            # "raw": True,  # assuming the image is in linear RGB
                            'format' : 'variant'
                        },
                        "scale": proj_brightness,
                        "to_world": mi.ScalarTransform4f().look_at(
                            origin=[0.005, 0, 0],  # along +X axis
                            target=[0, 0, 0],
                            up=[0, 0, 1],  # Z-up
                        ),
                        "fov": proj_fov,
                        "response_mode": proj_response_mode,
                    },
                    
                    "ambient": {
                        "type": "constant",
                        "radiance": {"type": "rgb", "value": ambient_color},
                    },
                }
    
    return scene_dict

In [ ]:
cam_wh

In [ ]:

@dr.wrap(source='torch', target='drjit')
def render_texture2(params, proj_tex_grad):



    params['projector.irradiance.data'] = proj_tex_grad 
    params.update()

    raw_render = mi.render(scene, params)



    return raw_render

def render(params, proj_tex_grad):

    raw_render = render_texture2(params,proj_tex_grad)


    alpha = raw_render[:, :, -1:]
    image = raw_render[:, :, :3]
    # no_alpha_render = gsoup.alpha_compose(render)
    image = tonemap_reinhard(image, exposure=1.0)
    # image = linear_to_srgb(image)
    final_image = add_alpha(image, alpha)

    final_image = final_image.clamp(0, 1.0)

    return final_image[:,:,:3]






In [ ]:
# black rectangle with white square in the center of size t
import copy 

scene_dict = generate_random_scene()
scene = mi.load_dict(scene_dict)
params = mi.traverse(scene)

t = 50
z = np.zeros((proj_wh[0], proj_wh[1], 3), dtype=np.float32)
oz = render(params, z).cpu()
anchor = np.zeros((proj_wh[0], proj_wh[1], 3), dtype=np.float32)
anchor[proj_wh[0]//2-t:proj_wh[0]//2+t, proj_wh[1]//2-t:proj_wh[1]//2+t, :] = 1
anchor = anchor.astype(np.float32)
o_orig = render(params, anchor).cpu()
plt.imshow(o_orig)
plt.show()
o = copy.deepcopy(o_orig)
o[proj_wh[0]//2-t:proj_wh[0]//2+t, proj_wh[1]//2-t:proj_wh[1]//2+t, :] = oz[proj_wh[0]//2-t:proj_wh[0]//2+t, proj_wh[1]//2-t:proj_wh[1]//2+t, :]

diff = (o - oz)**2
# create an heatmap of the difference and add a scale bar
plt.imshow(diff, cmap='hot')    
plt.colorbar()
plt.title('Difference Heatmap')
plt.show()

plt.imshow(o)
plt.show()
plt.imshow(oz)
plt.show()

In [ ]:
import torch
import torch.nn.functional as F
# Define glow parameters
glow_radius = 10  # Controls the spread of the glow
glow_strength = 10  # Controls the intensity of the glow
sigma = glow_radius / 6.0

def create_gaussian_kernel_2d(kernel_size, sigma):
    """Create a 2D Gaussian kernel using PyTorch"""
    coords = torch.arange(kernel_size, dtype=torch.float32) - kernel_size // 2
    g = torch.exp(-(coords ** 2) / (2 * sigma ** 2))
    g = g / g.sum()
    
    # Create 2D kernel
    kernel_2d = g.unsqueeze(0) * g.unsqueeze(1)
    return kernel_2d.unsqueeze(0).unsqueeze(0)

def gaussian_glow(image, kernel_size = glow_radius):
    """Apply Gaussian blur using PyTorch convolution with groups (no for loops)"""
    # Ensure kernel size is odd
    if kernel_size % 2 == 0:
        kernel_size += 1
    
    sigma = glow_radius / 6.0  # Adjust sigma based on radius
    # Create Gaussian kernel
    kernel = create_gaussian_kernel_2d(kernel_size, sigma)
    
    # Reshape image from (H, W, C) to (1, C, H, W) for conv2d
    img_4d = image.permute(2, 0, 1).unsqueeze(0)  # (1, 3, H, W)
    
    # Repeat kernel for each channel: (3, 1, kernel_size, kernel_size)
    kernel_3ch = kernel.repeat(3, 1, 1, 1)
    
    # Apply convolution with groups=3 (one kernel per channel)
    padding = kernel_size // 2
    blurred_4d = F.conv2d(img_4d, kernel_3ch, padding=padding, groups=3)
    
    # Convert back to (H, W, C) format
    img_blurred = blurred_4d.squeeze(0).permute(1, 2, 0)

    img_blended = image + img_blurred * glow_strength

    
    return img_blended

In [ ]:
# black rectangle with white square in the center of size t
import copy 



anchor_b = gaussian_glow(torch.tensor(anchor)).numpy()
anchor_b = anchor_b.astype(np.float32)
o_orig = render(params, anchor_b).cpu()
plt.imshow(o_orig)
plt.show()
o = copy.deepcopy(o_orig)
o[proj_wh[0]//2-t:proj_wh[0]//2+t, proj_wh[1]//2-t:proj_wh[1]//2+t, :] = oz[proj_wh[0]//2-t:proj_wh[0]//2+t, proj_wh[1]//2-t:proj_wh[1]//2+t, :]

diff = (o - oz)**2
# create an heatmap of the difference and add a scale bar
plt.imshow(diff, cmap='hot')    
plt.colorbar()
plt.title('Difference Heatmap')
plt.show()

plt.imshow(o)
plt.show()
plt.imshow(oz)
plt.show()

In [ ]:
torch.from_numpy(anchor).permute(2, 0, 1).shape

In [ ]:
# gaussian_blur_torch(torch.from_numpy(anchor).permute(2, 0, 1))

In [ ]:
# # black rectangle with white square in the center of size t
# import copy 

# scene_dict = generate_random_scene()
# scene = mi.load_dict(scene_dict)
# params = mi.traverse(scene)

# t = 50
# z = np.zeros((proj_wh[0], proj_wh[1], 3), dtype=np.float32)
# oz = render(params, z).cpu()
# anchor = np.zeros((proj_wh[0], proj_wh[1], 3), dtype=np.float32)
# anchor[proj_wh[0]//2-t:proj_wh[0]//2+t, proj_wh[1]//2-t:proj_wh[1]//2+t, :] = 1
# anchor = anchor.astype(np.float32)
# # anchor_blured = gaussian_blur_torch(torch.from_numpy(anchor).permute(2, 0, 1), sigma=1.0).squeeze(0).permute(1, 2, 0).numpy()
# anchor_blured = gaussian_blur_torch(torch.from_numpy(anchor))
# o_orig = render(params, anchor_blured).cpu()
# plt.imshow(o_orig)
# plt.show()
# o = copy.deepcopy(o_orig)
# o[proj_wh[0]//2-t:proj_wh[0]//2+t, proj_wh[1]//2-t:proj_wh[1]//2+t, :] = oz[proj_wh[0]//2-t:proj_wh[0]//2+t, proj_wh[1]//2-t:proj_wh[1]//2+t, :]

# diff = (o - oz)**2
# # create an heatmap of the difference and add a scale bar
# plt.imshow(diff, cmap='hot')    
# plt.colorbar()
# plt.title('Difference Heatmap')
# plt.show()

# plt.imshow(o)
# plt.show()
# plt.imshow(oz)
# plt.show()

# comp

asd

In [ ]:

@dr.wrap(source='torch', target='drjit')
def render_texture2(params, proj_tex_grad, scene):



    params['projector.irradiance.data'] = proj_tex_grad 
    params.update()

    raw_render = mi.render(scene, params)



    return raw_render

def render(params, proj_tex_grad, scene):

    raw_render = render_texture2(params,proj_tex_grad, scene)


    alpha = raw_render[:, :, -1:]
    image = raw_render[:, :, :3]
    # no_alpha_render = gsoup.alpha_compose(render)
    image = tonemap_reinhard(image, exposure=1.0)
    # image = linear_to_srgb(image)
    final_image = add_alpha(image, alpha)

    final_image = final_image.clamp(0, 1.0)

    return final_image[:,:,:3]






In [ ]:
def create_scene(proj_wh, cam_wh):
    """
    helper function creating a mitsuba scene with a projector-camera pair
    """
    projector_scene = gsoup.ProjectorScene()
    proj_cv_K = np.array(
        [
            [proj_wh[0], 0.0, proj_wh[0] / 2],
            [0.0, proj_wh[1], proj_wh[1] / 2],
            [0.0, 0.0, 1.0],
        ]
    )
    cam_cv_K = np.array(
        [
            [cam_wh[0], 0.0, cam_wh[0] / 2],
            [0.0, cam_wh[1], cam_wh[1] / 2],
            [0.0, 0.0, 1.0],
        ]
    )
    projector_scene.create_default_scene(
        proj_wh=proj_wh,
        cam_wh=cam_wh,
        cam_cv_K=cam_cv_K,
        proj_cv_K=proj_cv_K,
        # proj_fov=45.0,
        # cam_fov=45.0,
        proj_brightness=2.0,
        spp=256,
    )
    transform = (
        mi.ScalarTransform4f().look_at(
            origin=[0.0, 0.0, 0.0],  # along +X axis
            target=[-1, 0, 0],
            up=[0, 0, 1],  # Z-up
        ),
    )
    projector_scene.set_projector_transform(np.array(transform[0].matrix))
    projector_scene.set_camera_transform(np.array(transform[0].matrix))
    return projector_scene


def simulate_procam(patterns_to_project, scene_dict, synth_V=None):
    """
    helper function that simulates a video projector projecting some patterns onto a scene
    :param patterns_to_project: a dictionary of numpy images to project
    :param scene: a class containing a mitsuba scene (see procam.py)
    :param synth_V: a synthetic color mixing matrix per-pixel, simulating real artifacts created by the projector
    """
    captures = {}
    ### simulate procam ###
    for i, pattern_name in enumerate(sorted(patterns_to_project.keys())):
        pattern = patterns_to_project[pattern_name]

        scene = mi.load_dict(scene_dict)
        params = mi.traverse(scene)
        # scene.set_projector_texture(pattern)
        # capture = scene.capture(raw=True)
        capture = render(params, pattern, scene).cpu().numpy()
        if synth_V is not None:
            capture_expanded = capture[..., None]  # (H, W, 3, 1)
            mixed = np.matmul(synth_V, capture_expanded)  # (H, W, 3, 1)
            capture = capture_expanded[..., 0]  # remove singleton dimension → (H, W, 3)
        captures[pattern_name] = capture
    ### end simulate procam ###
    return captures

In [ ]:
create_scene(proj_wh, cam_wh)

In [ ]:
print("Photometric Calibration Example")
proj_wh = (512, 512)
cam_wh = (512, 512)
low_val = 80 / 255
high_val = 170 / 255
n_samples_per_channel = 20
################## offline steps for photometric calibration ##################
# 0. create a scene
scene_dict = generate_random_scene()#create_scene(proj_wh, cam_wh)
# 1. create patterns for calibration
# test_texture = gsoup.generate_voronoi_diagram(512, 512, 1000)
# test_texture = gsoup.to_float(test_texture)
patterns = {
    # "test_image": test_texture,
    "all_black": np.zeros(proj_wh + (3,), dtype=np.float32),
    "off_image": np.ones(proj_wh + (3,), dtype=np.float32) * low_val,
    "red_image": np.ones(proj_wh + (3,), dtype=np.float32) * low_val,
    "green_image": np.ones(proj_wh + (3,), dtype=np.float32) * low_val,
    "blue_image": np.ones(proj_wh + (3,), dtype=np.float32) * low_val,
    "on_image": np.ones(proj_wh + (3,), dtype=np.float32) * high_val,
    "white_image": np.ones(proj_wh + (3,), dtype=np.float32),
}
patterns["red_image"][:, :, 0] = high_val
patterns["green_image"][:, :, 1] = high_val
patterns["blue_image"][:, :, 2] = high_val
input_values = np.linspace(0.0, 1.0, num=20)
for i in range(n_samples_per_channel):
    patterns["gray_{:03d}".format(i)] = (
        np.ones(proj_wh + (3,), dtype=np.float32) * input_values[i]
    )
# 2. project patterns and acquire images (also simulate a global color mixing matrix)
# synth_V = np.array([[0.9, 0.1, 0.1], [0.2, 0.8, 0.2], [0.1, 0.1, 0.9]])
captured = simulate_procam(patterns, scene_dict, synth_V=None)
import matplotlib.pyplot as plt

for c in captured.keys():
    plt.imshow(captured[c])
    plt.title(c)
    plt.show()



In [ ]:

# 3. we use a "linear" camera here, if not possible we need to find camera response function per channel
# 4. and linearize camera response function
# 5. no need to white balance camera channels, color mixing matrix will take care of that
# 6. find inverse of color mixing matrix per-pixel
inv_v = gsoup.procam.estimate_color_mixing_matrix(
    off_image=captured["off_image"],
    red_image=captured["red_image"],
    green_image=captured["green_image"],
    blue_image=captured["blue_image"],
    cam_inv_response=None,
)
# 7. find projector inverse response function
measured = np.stack(
    [captured["gray_{:03d}".format(i)] for i in range(n_samples_per_channel)],
    axis=0,
)
proj_response = gsoup.procam.estimate_projector_inverse_response(
    measured,
    input_values=input_values,
    fg_mask=None,
)

In [ ]:
import glob
import numpy as np
import torchvision
resizer = torchvision.transforms.Resize((512, 512))
caltec_photos = '/home/dcor/niskhizov/caltech-101/101_ObjectCategories/'
ls = glob.glob(caltec_photos + '/*/*.jpg')
random_caltech_photo ='/home/dcor/niskhizov/caltech-101/101_ObjectCategories/bonsai/image_0098.jpg'# np.random.choice(ls)
texture = cv2.cvtColor(cv2.imread(random_caltech_photo), cv2.COLOR_BGR2RGB)#gsoup.generate_voronoi_diagram(512, 512, 1000)*0+255
proj_texture = gsoup.to_float(texture)
proj_tex = torch.from_numpy(proj_texture).cuda() 
proj_tex_r = resizer(proj_tex.permute(2, 0, 1)) * 0.6


In [ ]:
texture = proj_tex_r.cpu().permute(1,2,0).numpy()#gsoup.generate_voronoi_diagram(512, 512, 1000)
# reduce brightness for compensation to work
texture_float = gsoup.to_float(texture) * 0.75
# texture_float = np.ones((512, 512, 3), dtype=np.float32) * 0.5
# 2. compute compensation image
compensation_image = gsoup.procam.compute_compensation_image(
    texture_float,
    inv_v,
    cam_inverse_response=None,
    proj_inverse_response=proj_response,
)
# 3. project compensation image and uncompensated for comaprisons
result = simulate_procam(
    {"compensation_image": compensation_image, "texture_float": texture_float},
    scene_dict,
)

In [ ]:
scene = mi.load_dict(scene_dict)
params = mi.traverse(scene)
o = render(params,texture, scene)

In [ ]:
plt.imshow(texture)

In [ ]:
plt.imshow(o.cpu().numpy())

In [ ]:
import sys
sys.path.append('/home/dcor/niskhizov/AdversarialRendering/mitsuba')

In [ ]:
plt.imshow(texture_float)

In [ ]:
plt.imshow(result["compensation_image"])

In [ ]:
plt.imshow(result['texture_float'])